#   DSSS Method Implementation (by Sergei Popkov)

Load neccessary modules. Requirements:

1) Anaconda for Python 3 (http://www.anaconda.com);  
2) soundfile package (pip install soundfile);  
3) librosa package (for spectogram plotting).

In [ ]:
import IPython
import soundfile
import numpy as np
from math import floor
import matplotlib.pyplot as plt
import librosa
from librosa.display import specshow

Spectogram plotting function based on the course practice template.

In [ ]:
def show_spectrogram(audio_signal, sampling_rate, channel = 0):
    # Select one channel in case of stereo signal
    if len(audio_signal.shape) > 1:
        audio_signal = audio_signal[:, channel]
    win_length = int(0.025 * sampling_rate)
    hop_length = int(0.01 * sampling_rate)
    spectrogram = np.abs(librosa.stft(audio_signal, hop_length=hop_length, win_length=win_length))
    # Plotting the spectrogram:
    specshow(librosa.amplitude_to_db(spectrogram, ref=np.max), sr=sampling_rate, hop_length=hop_length, y_axis='linear', x_axis='time')
    plt.title('Spectrogram')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

PN generator

In [ ]:
# Generate password-related pseudo-noise (PN)
def password_to_PN(passw, length):
    
    # Password can't be empty and must contain at least 2 characters
    if not passw: passw=''
    if len(passw)<2: passw=passw.zfill(2)
    
    # One of the approaches to create the password-related seed for the pseudo-noise sequence generation
    p = np.array([ord(c) for c in passw])
    s = int(np.sum(p / np.max(p) * range(1, len(p) + 1)) * (10 ** min(len(p), 4)))
    
    # Generate the pseudo-noise sequence of given length for switching the phase;
    # return PN with the corresponding password
    rng = np.random.RandomState(s)
    return (passw, np.array([1 if r else -1 for r in rng.rand(length, 1) > 0.5]))

Global constants (both for encoding and extracting)

In [ ]:
# The minimal segment length
L_MIN = 8 * 1024

## 1. Encoding message

Parameters and constants

In [ ]:
# The carrier file name { <<"Ambient Wave 45" by Erokia>> from https://freesound.org/people/Erokia/sounds/482706/ }
CARRIER_FILENAME = "wave45.wav"

# The result file name
OUTPUT_FILENAME = "output.wav"

# The embedding strength (the power of the noise with the message over original content).
# The less the value, the less recognizable the noise is, yet the message may be decoded with errors or got lost.
ALPHA = 0.029

# The message to hide
MESSAGE = "Text to be hidden"

# The password
PASSWORD = "Tricky"

# Lower bound of mixed signal (-1 reverses ["flips"] the phase)
SMOOTH_LOWER = -1

# Upper bound of mixed signal (1 keeps the phase intact)
SMOOTH_UPPER = 1

#Number of points in the Hanning window (may be interpreted as a length to be smoothed)
SMOOTH_HANNING = 256

In [ ]:
# Not necessary for this project to work, but if the sound is not properly loaded by IPython 
# and this fact irritates you, uncomment and run once (already done for this project):

#signal, rate = soundfile.read(CARRIER_FILENAME)
#soundfile.write(CARRIER_FILENAME, signal, rate)

In [ ]:
IPython.display.Audio(CARRIER_FILENAME)

Load the carrier, check its properties, analyze the message and decide whether it's possible to store the whole message in a carrier.

In [ ]:
# Load the audio file data
signal, rate = soundfile.read(CARRIER_FILENAME)
# Audio length
a_len = signal.shape[0]
# Transform the message into bits, one byte per symbol ("letter")
bits=''.join([bin(ord(letter))[2:].zfill(8) for letter in MESSAGE])

# Number of bits
bits_len = len(bits)
# Segment (chip) length (not less than minimal)
L = max(L_MIN, floor(a_len / bits_len))
# Number of segments
N = floor(a_len / L)
# Number of segments should be aligned to bits per byte (symbol)
N -= N % 8
# Break the process in case of error (change the message or the carrier, then)
if bits_len > N:
    print('Not enough sound length to hide the message.')
    exit()
# Otherwise, keep received bits aligned with zeros from the right side to form the proper length
bits = ''.join((bits,'0' * (N - bits_len)))

#Spectrogram
show_spectrogram(signal, rate)

The signal smoothing: mixing message-related signal with the Hanning window.

In [ ]:
# Signal to spread data: each bit is being repeated L times
mixer_signal = np.array([int(i) for i in ''.join([i * L for i in bits])])
# Convolution of the Hanning window and the transformed signal
conv = np.convolve(mixer_signal, np.hanning(SMOOTH_HANNING))
# Normalization
HannHalf=int(SMOOTH_HANNING / 2)
norm = conv[HannHalf : -HannHalf + 1] / max(abs(conv))
# Bounds adjusting according to the parameters
result_mix = norm * (SMOOTH_UPPER - SMOOTH_LOWER) + SMOOTH_LOWER

Pseudo-noise (based on provided password) generation.

In [ ]:
# PN
actual_password, PN = password_to_PN(PASSWORD, L)
# Warning in case of invalid (not suitable for PN generation) password
if actual_password != PASSWORD:
    print("Warning: provided password is not valid, another one has been applied:", actual_password)
# Repeat the sequence for each symbol
preprocessed_PN = np.repeat(PN, N)

Finally, embed message and store the result into output file.

In [ ]:
# We need to modify just one channel, while the carrier itself may be mono or stereo.
# It is possible to use parameter "always_2d = True" while loading the file, but then its structure would be changed
# upon saving, which is suspicious. Thus, it's better to detect it manually.
# Also, we use only a part of the signal (N * L, where N is a number of segments, L is a segment length).
# Therefore, to properly decode, we need to know password and original message length.

# Internally, the sequences in Python are represented as pointers, that's why we can do this ...
signal_part = signal[: L * N, 0] if len(signal.shape) > 1 else signal[: L * N]
# ...and still modify the original signal, avoiding the code duplication this way.
signal_part += ALPHA * result_mix * preprocessed_PN
# Store the result.
soundfile.write(OUTPUT_FILENAME, signal, rate)
# Write the output info necessary to extract the message
print(" File name: ", OUTPUT_FILENAME, "\n Message length: ", str(bits_len), "\n Password: ", actual_password)

## 2. Extracting message

This part may be used independently. Here, it is used to check whether the message was properly embedded (in other words, the embedding strength was powerful enough to recover message intact later).

Parameters and constants (here, they are equal to certain values already known from the code above).

In [ ]:
# The audio file with encoded message
STENO_FILENAME = OUTPUT_FILENAME

#The message length
MESSAGE_LENGTH = bits_len

# The password
PASSWORD = actual_password

Load the signal for further processing

In [ ]:
IPython.display.Audio(STENO_FILENAME)

In [ ]:
# PN
_, PN = password_to_PN(PASSWORD, L)
# Repeat the sequence for each symbol
preprocessed_PN = np.repeat(PN, N)
# Open the file
signal, _ = soundfile.read(STENO_FILENAME, always_2d = True)
# Segment length (not less than minimal)
L = max(L_MIN, floor(a_len / MESSAGE_LENGTH))
# Number of segments
N = floor(a_len / L)
# Number of segments should be aligned to bits per byte (symbol)
N -= N % 8

#Spectrogram
show_spectrogram(signal, rate)

Load the bits of text: count the average phase inversion per chip for every encoded bit in regards of the password (represented as a pseudo-noise). Get corresponding bit values as a result.

In [ ]:
bits_received=['0' if i < 0 else '1' for i in [sum(signal[L * i: L * (i + 1), 0] * preprocessed_PN[L * i: L * (i + 1)]) / L for i in range(N)]]

Perform message extraction.

In [ ]:
# This lambda function converts the bit string into proper character (Input: string -> char code -> Output: char)
symbol_from_bitseq = lambda y : chr(int(''.join(y), 2))
# Perform the transformation by function described above for every byte (8 bits) in the bit sequence
decoded_symbols = [symbol_from_bitseq(bits_received[i : i + 8]) for i in range(0, len(bits_received), 8)]
# Gather together all symbols (characters) to finally receive the whole message
decoded_text = ''.join(decoded_symbols)
# Show the result
decoded_text

## 3. Validation

Check if the message is decoded properly; if it is not, show the error.

In [ ]:
print("""Message is embedded successfully!
The result file can be sent to the receiver who knows the code.""" if (decoded_text == MESSAGE) else """Message was not embedded correctly.
Try changing the alpha (embedding strength) parameter.""")